# Importing Necessary Packages

In [33]:
%load_ext autoreload
%autoreload 2
    
# Importing all the functions I made for data creation/manipulation
from data_creation import data_creation, pickle_me, get_pickle

# Model Building
import pandas as pd
import numpy as np

# Neural Network Building
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import regularizers
from keras.layers import Dense, Flatten, Dropout, Conv1D, Conv2D, ELU

# Classification Metrics
from sklearn.metrics import confusion_matrix
from keras.metrics import Precision, Recall
from data_creation import get_precisions
from sklearn.metrics import precision_score, recall_score, f1_score

# Packaging Data
import pickle

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Data Creation

#### I only need to run this once to create my data. It will pickle my data for future reference.
I will only run the categories object since I call on it later in the notebook.

In [2]:
# Directory where all the images are stored.
#data_dir = 'C://Users/Cristian/Documents/flatiron/Capstone/Fruit'

# Categories of Fruit with the correct labels.
categories = ['Apple', 'Banana', 'Carambola', 'Guava', 'Kiwi', 'Mango',
              'Muskmelon', 'Orange', 'Peach', 'Pear', 'Persimmon', 'Pitaya',
              'Plum', 'Pomegranate', 'Tomato']

##### Creating Grayscale data and pickling it

In [3]:
#X_gray, y_gray = data_creation(50, 0, categories, data_dir)
#pickle_me(X_gray,y_gray, 'X_grayscale', 'y_grayscale')

##### Creating Color data and pickling it

In [4]:
#X_color, y_color = data_creation(50, 1, categories, data_dir)
#pickle_me(X_color, y_color,'X_color', 'y_color')

##### Getting our pickled data for grayscale images and color images

In [5]:
X_grayscale, y_grayscale = get_pickle('X_grayscale.pickle', 'y_grayscale.pickle')

In [6]:
X_rgb, y_rgb = get_pickle('X_color.pickle', 'y_color.pickle')

# Data Manipulation

#### Creating a train, test split for cross-validation.
'gs' will be for grayscale images, 'rgb' will be for color images.

I will rescale by 255 as a standard procedure for all my images and reshape
them to the correct input I need.

I need a target input shape of (15,) instead of (1,), which is why I use
get_dummies. This will create a column for each label with entries as a binary
input.

In [42]:
X_rgb_sc = X_rgb/255.0
X_rgb_reshape = X_rgb_sc.reshape(X_rgb.shape[0],50,50,3)

In [43]:
X_gs_sc = X_grayscale/255.0
X_gs_reshape = X_gs_sc.reshape(X_grayscale.shape[0],50,50,)

In [47]:
X_gs_train, X_gs_test, y_gs_train, y_gs_test = train_test_split(X_gs_reshape,
                                                                y_grayscale, 
                                                                test_size=.2,
                                                                random_state=42)

In [48]:
X_rgb_train, X_rgb_test, y_rgb_train, y_rgb_test = train_test_split(X_rgb_reshape,
                                                                    y_rgb, 
                                                                    test_size=.2,
                                                                    random_state=42)

In [49]:
y_gs_train_dummies = np.array(pd.get_dummies(y_gs_train))
y_gs_test_dummies = np.array(pd.get_dummies(y_gs_test))

y_rgb_train_dummies = np.array(pd.get_dummies(y_rgb_train))
y_rgb_test_dummies = np.array(pd.get_dummies(y_rgb_test))

# Model Building
As a way to better visualize how models are doing compared to one another, I will create the neural networks, compile them, and fit our data. Then, I will have them predict classes and put our classification metrics into a dataframe. You will notice that these models are very simple. I had already tried adding more layers without much progress. Keeping the model to only a few layers big could achieve similar or even better results that a more complex and larger model.

## Grayscale Models

### Model 1
First Simple Model
Im using a Flatten layer so I can input my data through a Dense layer where softmax will split into the 15 classifications I have for the
fruit.

In [50]:
model_1 = Sequential([Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_1.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_1_results = model_1.fit(X_gs_train, y_gs_train_dummies, epochs=5, validation_data =(X_gs_test, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 8s 213us/step - loss: 1.7784 - categorical_accuracy: 0.5349 - precision_13: 0.8827 - recall_13: 0.2061 - val_loss: 1.3975 - val_categorical_accuracy: 0.6249 - val_precision_13: 0.8128 - val_recall_13: 0.3607
Epoch 2/5
35524/35524 [==============================] - 3s 73us/step - loss: 1.2330 - categorical_accuracy: 0.6814 - precision_13: 0.9476 - recall_13: 0.4472 - val_loss: 1.1780 - val_categorical_accuracy: 0.7054 - val_precision_13: 0.9462 - val_recall_13: 0.4694
Epoch 3/5
35524/35524 [==============================] - 2s 70us/step - loss: 1.1188 - categorical_accuracy: 0.7100 - precision_13: 0.9543 - recall_13: 0.5273 - val_loss: 1.1452 - val_categorical_accuracy: 0.6961 - val_precision_13: 0.9337 - val_recall_13: 0.5263
Epoch 4/5
35524/35524 [==============================] - 2s 67us/step - loss: 1.0565 - categorical_accuracy: 0.7235 - precision_13: 0.9589 - recall_13: 0.5677

### Model 2
Adding a convolutional layer to try to improve performance.

In [51]:
model_2 = Sequential([Conv1D(100, kernel_size=(1), activation='relu', input_shape=(50,50)),
                      Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_2.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_2_results = model_2.fit(X_gs_train, y_gs_train_dummies, epochs=5, validation_data =(X_gs_test, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 5s 142us/step - loss: 1.1257 - categorical_accuracy: 0.6914 - precision_14: 0.9408 - recall_14: 0.5381 - val_loss: 1.0079 - val_categorical_accuracy: 0.7184 - val_precision_14: 0.9030 - val_recall_14: 0.6303
Epoch 2/5
35524/35524 [==============================] - 5s 143us/step - loss: 0.8033 - categorical_accuracy: 0.7750 - precision_14: 0.9603 - recall_14: 0.6915 - val_loss: 0.8260 - val_categorical_accuracy: 0.7653 - val_precision_14: 0.9555 - val_recall_14: 0.6861
Epoch 3/5
35524/35524 [==============================] - 5s 141us/step - loss: 0.7365 - categorical_accuracy: 0.7891 - precision_14: 0.9646 - recall_14: 0.7129 - val_loss: 0.7781 - val_categorical_accuracy: 0.7814 - val_precision_14: 0.9453 - val_recall_14: 0.7137
Epoch 4/5
35524/35524 [==============================] - 5s 145us/step - loss: 0.6982 - categorical_accuracy: 0.7974 - precision_14: 0.9682 - recall_14: 0.7

### Model 3
Changing the number of neurons for the convolutional layer to see if less neurons can have an improvement.

In [18]:
model_3 = Sequential([Conv1D(50, kernel_size=(1), activation='relu', input_shape=(50,50)),
                      Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_3.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_3_results = model_3.fit(X_gs_train, y_gs_train_dummies, epochs=5, validation_data =(X_gs_test, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 4s 126us/step - loss: 1.2043 - categorical_accuracy: 0.6723 - precision_4: 0.9382 - recall_4: 0.4944 - val_loss: 0.9893 - val_categorical_accuracy: 0.7280 - val_precision_4: 0.9491 - val_recall_4: 0.6104
Epoch 2/5
35524/35524 [==============================] - 4s 120us/step - loss: 0.8943 - categorical_accuracy: 0.7543 - precision_4: 0.9523 - recall_4: 0.6561 - val_loss: 0.9696 - val_categorical_accuracy: 0.7367 - val_precision_4: 0.9072 - val_recall_4: 0.6515
Epoch 3/5
35524/35524 [==============================] - 4s 121us/step - loss: 0.8234 - categorical_accuracy: 0.7701 - precision_4: 0.9567 - recall_4: 0.6855 - val_loss: 1.0115 - val_categorical_accuracy: 0.7123 - val_precision_4: 0.8481 - val_recall_4: 0.6348
Epoch 4/5
35524/35524 [==============================] - 4s 121us/step - loss: 0.7838 - categorical_accuracy: 0.7780 - precision_4: 0.9585 - recall_4: 0.6992 - val_loss

### Model 4
Adding another dense layer with another activation function. I had tested out several activation functions that all tended to
either stay stagnant or decrease performance.

In [19]:
model_4 = Sequential([Conv1D(50, kernel_size=(1), activation='relu', input_shape=(50,50)),
                      Flatten(input_shape=(50,50)),
                      Dense(30, activation='sigmoid'),
                      Dense(15, activation='softmax')])
model_4.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_4_results = model_4.fit(X_gs_train, y_gs_train_dummies, epochs=5, validation_data =(X_gs_test, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 5s 130us/step - loss: 1.6367 - categorical_accuracy: 0.5986 - precision_5: 0.9938 - recall_5: 0.1446 - val_loss: 1.3592 - val_categorical_accuracy: 0.6944 - val_precision_5: 0.9902 - val_recall_5: 0.2959
Epoch 2/5
35524/35524 [==============================] - 5s 133us/step - loss: 1.1800 - categorical_accuracy: 0.7230 - precision_5: 0.9864 - recall_5: 0.4538 - val_loss: 1.1267 - val_categorical_accuracy: 0.7277 - val_precision_5: 0.9876 - val_recall_5: 0.5095
Epoch 3/5
35524/35524 [==============================] - 5s 136us/step - loss: 1.0331 - categorical_accuracy: 0.7457 - precision_5: 0.9819 - recall_5: 0.5692 - val_loss: 1.0417 - val_categorical_accuracy: 0.7273 - val_precision_5: 0.9807 - val_recall_5: 0.5673
Epoch 4/5
35524/35524 [==============================] - 5s 129us/step - loss: 0.9521 - categorical_accuracy: 0.7584 - precision_5: 0.9803 - recall_5: 0.6164 - val_loss

### Model 5
Adding a dropout layer. This would make sure that we can prevent our model from overrelying on certain features to make predictions.

In [20]:
model_5 = Sequential([Conv1D(50, kernel_size=(1), activation='relu', input_shape=(50,50)),
                      Flatten(input_shape=(50,50)),
                      Dropout(0.2),
                      Dense(30, activation='sigmoid'),
                      Dense(15, activation='softmax')])
model_5.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_5_results = model_5.fit(X_gs_train, y_gs_train_dummies, epochs=5, validation_data =(X_gs_test, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 6s 166us/step - loss: 1.6997 - categorical_accuracy: 0.5646 - precision_6: 0.9820 - recall_6: 0.1245 - val_loss: 1.3655 - val_categorical_accuracy: 0.6710 - val_precision_6: 0.9808 - val_recall_6: 0.2990
Epoch 2/5
35524/35524 [==============================] - 6s 158us/step - loss: 1.2623 - categorical_accuracy: 0.6953 - precision_6: 0.9726 - recall_6: 0.3859 - val_loss: 1.1646 - val_categorical_accuracy: 0.7118 - val_precision_6: 0.9695 - val_recall_6: 0.4696
Epoch 3/5
35524/35524 [==============================] - 6s 160us/step - loss: 1.1165 - categorical_accuracy: 0.7241 - precision_6: 0.9716 - recall_6: 0.5085 - val_loss: 1.1108 - val_categorical_accuracy: 0.7093 - val_precision_6: 0.9602 - val_recall_6: 0.5240
Epoch 4/5
35524/35524 [==============================] - 6s 158us/step - loss: 1.0327 - categorical_accuracy: 0.7387 - precision_6: 0.9688 - recall_6: 0.5688 - val_loss

## Color Images Model

### Model 6
We are starting to use color images since color is a big factor in determing fruit apart. 

Model 6 will be first simple model for color images.

In [59]:
model_6 = Sequential([Conv2D(50, kernel_size=(1), activation='relu'),
                      Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_6.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_6_results = model_6.fit(X_rgb_train, y_rgb_train_dummies, epochs=5, validation_data =(X_rgb_test, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 65s 2ms/step - loss: 1.0895 - categorical_accuracy: 0.8270 - precision_15: 0.8979 - recall_15: 0.7671 - val_loss: 0.4591 - val_categorical_accuracy: 0.8479 - val_precision_15: 0.8956 - val_recall_15: 0.8040
Epoch 2/5
35524/35524 [==============================] - 54s 2ms/step - loss: 0.2556 - categorical_accuracy: 0.9208 - precision_15: 0.9584 - recall_15: 0.8861 - val_loss: 0.2882 - val_categorical_accuracy: 0.9037 - val_precision_15: 0.9325 - val_recall_15: 0.8790
Epoch 3/5
35524/35524 [==============================] - 54s 2ms/step - loss: 0.1938 - categorical_accuracy: 0.9422 - precision_15: 0.9701 - recall_15: 0.9128 - val_loss: 0.2265 - val_categorical_accuracy: 0.9218 - val_precision_15: 0.9492 - val_recall_15: 0.8998
Epoch 4/5
35524/35524 [==============================] - 54s 2ms/step - loss: 0.1604 - categorical_accuracy: 0.9537 - precision_15: 0.9753 - recall_15: 0.9287 

### Model 7

Changing the activation function to see if there is a difference and lowering the neurons.

In [24]:
model_7 = Sequential([Conv2D(50, kernel_size=(1), activation='sigmoid'),
                      Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_7.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_7_results = model_7.fit(X_rgb_train, y_rgb_train_dummies, epochs=5, validation_data =(X_rgb_test, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 56s 2ms/step - loss: 47.8913 - categorical_accuracy: 0.1101 - precision_9: 0.1109 - recall_9: 0.1052 - val_loss: 17.0118 - val_categorical_accuracy: 0.0740 - val_precision_9: 0.0740 - val_recall_9: 0.0740
Epoch 2/5
35524/35524 [==============================] - 56s 2ms/step - loss: 6.8147 - categorical_accuracy: 0.2680 - precision_9: 0.2938 - recall_9: 0.2401 - val_loss: 10.8743 - val_categorical_accuracy: 0.0825 - val_precision_9: 0.0810 - val_recall_9: 0.0804
Epoch 3/5
35524/35524 [==============================] - 56s 2ms/step - loss: 3.5527 - categorical_accuracy: 0.3965 - precision_9: 0.4541 - recall_9: 0.3511 - val_loss: 8.1977 - val_categorical_accuracy: 0.1663 - val_precision_9: 0.1640 - val_recall_9: 0.1621 precision_9: 0.4534 - recall_9: 
Epoch 4/5
35524/35524 [==============================] - 58s 2ms/step - loss: 2.3456 - categorical_accuracy: 0.4912 - precision_9: 0.57

### Model 8

Adding a dropout layer for the same reason as before. Lowering number of neurons as well. My valudation values for accuracy, precision, and recall seemed to suffer after 5 epocs, so I lowered it to 4.

In [86]:
model_8 = Sequential([Conv2D(85, kernel_size=(1), activation='relu'),
                      Flatten(input_shape=(50,50)),
                      Dropout(0.2),
                      Dense(15, activation='softmax')])
model_8.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_8_results = model_8.fit(X_rgb_train, y_rgb_train_dummies, epochs=4, validation_data =(X_rgb_test, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/4
35524/35524 [==============================] - 208s 6ms/step - loss: 1.2635 - categorical_accuracy: 0.8364 - precision_19: 0.9006 - recall_19: 0.7818 - val_loss: 0.3063 - val_categorical_accuracy: 0.9051 - val_precision_19: 0.9499 - val_recall_19: 0.8660
Epoch 2/4
35524/35524 [==============================] - 187s 5ms/step - loss: 0.2267 - categorical_accuracy: 0.9288 - precision_19: 0.9610 - recall_19: 0.8980 - val_loss: 0.2590 - val_categorical_accuracy: 0.9144 - val_precision_19: 0.9382 - val_recall_19: 0.8890
Epoch 3/4
35524/35524 [==============================] - 187s 5ms/step - loss: 0.1680 - categorical_accuracy: 0.9501 - precision_19: 0.9727 - recall_19: 0.9251 - val_loss: 0.2549 - val_categorical_accuracy: 0.9130 - val_precision_19: 0.9385 - val_recall_19: 0.8932
Epoch 4/4
35524/35524 [==============================] - 185s 5ms/step - loss: 0.1362 - categorical_accuracy: 0.9614 - precision_19: 0.9789 - recall_19: 0.9

#### Model 9
Adding bias regularizers and lowering the neurons for the convolution layer.

In [26]:
model_9 = Sequential([Conv2D(10, kernel_size=(1), activation='relu'),
                      Flatten(input_shape=(50,50)),
                      Dropout(0.2),
                      Dense(64, activation='sigmoid', bias_regularizer=regularizers.l1_l2(l1=0.02,l2=0.02)),
                      Dense(15, activation='softmax')])
model_9.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_9_results = model_9.fit(X_rgb_train, y_rgb_train_dummies, epochs=5, validation_data =(X_rgb_test, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 42s 1ms/step - loss: 2.6672 - categorical_accuracy: 0.1123 - precision_11: 0.0000e+00 - recall_11: 0.0000e+00 - val_loss: 2.6440 - val_categorical_accuracy: 0.1133 - val_precision_11: 0.0000e+00 - val_recall_11: 0.0000e+00
Epoch 2/5
35524/35524 [==============================] - 41s 1ms/step - loss: 2.1803 - categorical_accuracy: 0.4173 - precision_11: 0.0000e+00 - recall_11: 0.0000e+00 - val_loss: 1.8460 - val_categorical_accuracy: 0.6127 - val_precision_11: 0.0000e+00 - val_recall_11: 0.0000e+00
Epoch 3/5
35524/35524 [==============================] - 40s 1ms/step - loss: 1.5132 - categorical_accuracy: 0.6906 - precision_11: 0.9992 - recall_11: 0.1121 - val_loss: 1.4231 - val_categorical_accuracy: 0.6791 - val_precision_11: 0.9874 - val_recall_11: 0.1582
Epoch 4/5
35524/35524 [==============================] - 42s 1ms/step - loss: 1.1553 - categorical_accuracy: 0.7627 - precision

### Model 10
Similar to model 8, but I'm changing the order of the layers and added a regularizer.

In [27]:
model_10 = Sequential([Conv2D(85, kernel_size=(1), activation='relu'),
                       Dropout(0.2),
                       Flatten(input_shape=(50,50)),
                       Dense(15, activation='softmax', activity_regularizer=regularizers.l1(0.02))])
model_10.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_10_results = model_10.fit(X_rgb_train, y_rgb_train_dummies, epochs=5, validation_data =(X_rgb_test, y_rgb_test_dummies)))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 184s 5ms/step - loss: 1.7627 - categorical_accuracy: 0.8407 - precision_12: 0.9064 - recall_12: 0.7809 - val_loss: 0.9211 - val_categorical_accuracy: 0.9117 - val_precision_12: 0.9501 - val_recall_12: 0.8710
Epoch 2/5
35524/35524 [==============================] - 181s 5ms/step - loss: 0.8636 - categorical_accuracy: 0.9315 - precision_12: 0.9618 - recall_12: 0.8998 - val_loss: 0.8660 - val_categorical_accuracy: 0.9268 - val_precision_12: 0.9551 - val_recall_12: 0.9006
Epoch 3/5
35524/35524 [==============================] - 181s 5ms/step - loss: 0.8081 - categorical_accuracy: 0.9507 - precision_12: 0.9726 - recall_12: 0.9260 - val_loss: 0.9580 - val_categorical_accuracy: 0.8892 - val_precision_12: 0.9083 - val_recall_12: 0.8707
Epoch 4/5
35524/35524 [==============================] - 182s 5ms/step - loss: 0.7780 - categorical_accuracy: 0.9611 - precision_12: 0.9784 - recall_12: 0.9

# Getting Metrics for my top 2 Models

My goal is to get the most amount of correct predictions. This means that of all the predictions I make, I want to get the most amount possible correct. This is similar to the precision metric for classification. Thus, I will first look at how these models do with precision. I will still look at recall and f1-score to get a sense on how these models are doing.

### Closer Look

In [88]:
model_8_predictions = model_8.predict_classes(X_rgb_test)
model_10_predictions = model_10.predict_classes(X_rgb_test)

### Precision

In [89]:
model_8_precision = precision_score(y_rgb_test, model_8_predictions, average='weighted')
model_10_precision = precision_score(y_rgb_test, model_10_predictions, average='weighted')
print('Model 8', model_8_precision)
print('Model 10:', model_10_precision)

Model 8 0.9355741893382752
Model 10: 0.940517600394757


### Recall

In [90]:
model_8_recall = recall_score(y_rgb_test, model_8_predictions, average='weighted')
model_10_recall = recall_score(y_rgb_test, model_10_predictions, average='weighted')
print('Model 8:', model_8_recall)
print('Model 10:', model_10_recall)

Model 8: 0.9327854086917361
Model 10: 0.9386399459581175


### F1-Score

In [91]:
model_8_f1 = f1_score(y_rgb_test, model_8_predictions, average='weighted')
model_10_f1 = f1_score(y_rgb_test, model_10_predictions, average='weighted')
print('Model 6:', model_8_f1)
print('Model 10:', model_10_f1)

Model 6: 0.9335625703486898
Model 10: 0.9390828291181957


## Conclusion

Model 8 and Model 10 are our top 2 models based on values in validation accuracy, precision, recall, and f1-score. However, I am choosing Model 8 as my main model even though the precision isn't as high, but still very similar to Model 10. They each have very similar scores in everything except loss. When it comes to loss, model 8 has a much lower crossentropy loss. This means the distribution of model 8's predictions are closer to the distribution of the actual labels.

#### Save my models
This will save each model architecture as a .json file with the weights of the features as a .h5 file. This will let me save all of my models to save me time in the future.

In [92]:
for model in models:
    i = models.index(model) + 1
    model_json = model.to_json()
    with open(f"Models/model{i}.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(f"Models/model_weights_{i}.h5")

#### Get Scores for all of my Data
I am using a weighted average since it takes into account the slight imbalance of the data.

In [96]:
m8_final_preds = model_8.predict_classes(X_rgb_reshape)
prec = precision_score(y_rgb, m8_final_preds, average='weighted')
rec = recall_score(y_rgb, m8_final_preds, average='weighted')
f1 = f1_score(y_rgb, m8_final_preds, average='weighted')
print('Precision:', prec,
      '\nRecall:', rec,
      '\nF1-Score:', f1)

Precision: 0.9650290606541851 
Recall: 0.9638562356438319 
F1-Score: 0.9641525579819867


## Packaging Predictions for EDA

In [97]:
predictions_df = pd.DataFrame(list(zip(y_rgb, m8_final_preds)))
predictions_df.columns = ['Actual', 'Predicted']

In [98]:
predictions_pickle_out = open('model_predictions.pickle', 'wb')
pickle.dump(predictions_df, predictions_pickle_out)
predictions_pickle_out.close()